# Bid Winner
add explantation in getting messy data to the salesforce ecosystem

All data contained in this repository, as well as any information queried or conveyed through its use, is publicly accessible and readily available on the internet. No proprietary, confidential, or restricted data is included or utilized.

## Imports
and some configuration

In [ ]:
import sys
sys.path.insert(0, '../scripts')
from govspend_search import GoveSpendSearch
from rfp_parser import RFPParse
from flis_search import FlisSearch
from parts_base import PartsBase
import pandas as pd
import os

search_id = "679aba32c9c299c7b31ade4f"
PATH_TO_DLL = "C:\\Users\\jackmchugh\\Downloads\\PublogDVD\\TOOLS\\MS12\\DecompDl64.dll"
PATH_TO_FEDLOG = "C:\\Users\\jackmchugh\\Downloads\\PublogDVD"

output_dir = "./output_files"
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, "target_list.csv")
output_manufacturer_file = os.path.join(output_dir, "manufacturer_information.csv")


## Helper Classes

In [5]:
pb = PartsBase()
govspend = GoveSpendSearch()
flis = FlisSearch(path_to_dll=PATH_TO_DLL, path_to_fedlog=PATH_TO_FEDLOG)
rfp_parser = RFPParse()

FileNotFoundError: Could not find module 'C:\Users\jmchugh\Downloads\PublogDVD\TOOLS\MS12\DecompDl64.dll' (or one of its dependencies). Try using the full path with constructor syntax.

## Process Data

### Run GovSpend Search

In [ ]:
govspend_result = govspend.search(search_id)
combined_result_list = []
manufacturer_result_list = []

### Process Bids

In [ ]:
count = 0

for bid in govspend_result['result']:
    parse_result = rfp_parser.parse_rfp(bid['description'])
    flis_result = flis.get_part_number_and_description(parse_result["niin"])

    ref_pn = (
        parse_result.get("part_number", "").strip()
        or flis_result.get("part_number", "").split(",")[0].strip()
        or "NO PART NUMBER"
    )

    num_pb_results = pb.get_live_qty(ref_pn)

    result = {
        "bid_number": bid["bidNumber"],
        "ref_pn": ref_pn,
        "parse_desc": parse_result["description"],
        "qty_requested": parse_result["quantity"],
        "num_qty_on_pb": num_pb_results,
        "approved_source": parse_result["approved_source"],
        "lead_time": parse_result["lead_time"],
        "incumbent": parse_result["incumbent"],
        "multiple_award": parse_result["multiple_award"],
        "approved_source_codes": parse_result["approved_source_codes"],
        "date_create": bid["postedDate"],
        "date_due": bid["dueDate"],
        "psc_code": bid["PSCCode"],
        "set_aside": bid["setAside"],
        "NSN": parse_result["nsn"],
        "flis_pn": flis_result["part_number"],
        "parse_pn": parse_result["part_number"],
        "flis_desc": parse_result["description"]
    }

    print(result)
    count += 1
    print(f"Number of BIDs processed", count)
    combined_result_list.append(result)

    manufacturer_data = flis.get_cage_codes_and_pricing(parse_result["niin"])
    if not manufacturer_data.empty:
        manufacturer_result_list.append(manufacturer_data)

### Save the Results

In [ ]:
df = pd.DataFrame(combined_result_list)
df.to_csv(output_file, index=False)
print(f"Data saved to {output_file}")

if manufacturer_result_list:
    df_manufacturer = pd.concat(manufacturer_result_list, ignore_index=True)
    df_manufacturer.to_csv(output_manufacturer_file, index=False)
    print(f"Manufacturer data saved to {output_manufacturer_file}")